In [1]:
numeric_stats = ['derived_fg', 'derived_ft', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
fantasy_stats = ['d_FG%', 'd_FT%', 'FG%', 'FT%','TRB', 'AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']
negative_stats = ['FTA', 'TOV', 'FGA']

In [2]:
# Import stats CSV
import pandas as pd

arr = pd.read_csv('season_stats.csv').set_index('Player')
# arr = arr.drop(['Rk', 'GS', 'Age', 'Pos', 'FG%', 'FT%', '2P%', 'eFG%', '3PA', '3P%', '2P', '2PA', 'Tm'], axis=1)
# arr = arr.groupby(arr['Player']).sum()
# arr['FT%'] = arr['FT'] * 100 / arr['FTA']
# arr['FG%'] = arr['FG'] * 100 / arr['FGA']
# arr['d_FG%'] = arr['FG'] * (arr['FG%'] - arr['FG%'].mean())
# arr['d_FT%'] = arr['FT'] * (arr['FT%'] - arr['FT%'].mean())
# # arr['TOV'] = -1 * arr['TOV']
# arr[['TRB', 'AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']] = arr[['TRB', 'AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']].div(arr['G'], axis=0)

In [3]:
# Import average draft values TSV
raw_draft_values = pd.read_csv('draft_values.tsv', sep='\t')
raw_draft_values
def transform_name(name):
    notes_loc = name.find('Notes')
    if notes_loc > -1:
        name = name[notes_loc+5:]
    note_loc = name.find('Note')
    if note_loc > -1:
        name = name[note_loc+4:]
    names = name.split()
    ret = names[0:2]
    if (names[2] == 'Jr.' or names[2] == 'Sr.'):
        ret = names[0:3]
    return ' '.join(ret)

def transform_average(val):
    val = val[1:]
    try:
        num_val = float(val)
        return num_val
    except ValueError:
        return 0

raw_draft_values['Name'] = raw_draft_values['Name'].transform(transform_name)
raw_draft_values['Avg $'] = raw_draft_values['Average'].transform(transform_average)
raw_draft_values = raw_draft_values.set_index('Name')
arr = arr.join(raw_draft_values[['Avg $']])
arr = arr.fillna(0)

In [7]:
arr['TOV'] = -1 * arr['TOV']
arr[fantasy_stats+['Avg $']].corr().to_json('./correlations.json')
arr['TOV'] = -1 * arr['TOV']

In [9]:
arr['Name'] = arr.index
arr[fantasy_stats+['Name','Avg $']].to_json('./player_stats.json', orient='records')